# Analyse Hookean model


In [1]:
import numpy as np
from matsindy.feature_library import FeatureLibrary

## Datasets selection

In [2]:
# Name: file
dataset_files = {
    'random_0_n1000': 'outputs/Hookean/random_0_n1000.npz',
    'random_2_n1000': 'outputs/Hookean/random_2_n1000.npz'
}

In [38]:
# Load data
dataset = {}
for name, file in dataset_files.items():
    with np.load(file, allow_pickle=True) as data:
        temp = dict(data)
        # Rename variables
        temp['S'] = temp.pop('S_average')
        temp['A'] = temp.pop('A_average')
        temp['∇U'] = temp.pop('gradU')
        temp['∇Uᵀ'] = np.transpose(temp['∇U'], axes=(0, 2, 1))
        # Save
        dataset[name] = temp

## Make library of features

Matrix libraries will take data in the form of a dictionary and access the variables using their name. Therefore we need to specify which names we want to involve in the library by defining `variable_names`. Another useful parameter is the `transpose_map` dictionary where we write information about the transpose of the variables. In this dictionary, a key is a variable name of the dataset, and the value is:
- Same name if the variable is symmetric;
- Same name with a leading `-` if the variable is skew;
- The name of the transpose if it is present in the dataset;
- `None` otherwise.

Here we will be using the stress `'S'` which is symmetric, and both the velocity gradient `'∇U'` and its transposed `'∇Uᵀ'`.

In [30]:
variable_names = {'S', '∇U', '∇Uᵀ'}
transpose_map = {'S':'S', '∇U':'∇Uᵀ', '∇Uᵀ':'∇U'}

Now we can create a new library containing polynomial terms by calling `from_polynomial_matrices`. We need to provide: 
1. `n_terms`: number of mutliplication terms.
2. `intercept`: include the identity matrix as a feature.
3. `symmetry`: symmetry property of the feature. Here we want only symmetric features.

In [31]:
library = FeatureLibrary.from_polynomial_matrices(variable_names=variable_names, 
                                                  transpose_map=transpose_map, 
                                                  n_terms=4, intercept=True, symmetry='symmetric')
#library

We will also make a scalar library which returns the trace of these polynomial terms. Note that trivially equal features have been removed.

In [32]:
library_trace = FeatureLibrary.from_polynomial_traces(variable_names, transpose_map, n_terms=4, intercept=False)
#library_trace

Before we go on, we will remove `tr(∇U)` from the library as we expect this should be null from imcompressibility assumption.

In [33]:
library_trace.remove_by_name('tr(∇U)')

Type of feature output: scalar
Variables names: {'S', '∇U', '∇Uᵀ'}
Variables transpose map: {'S': 'S', '∇U': '∇Uᵀ', '∇Uᵀ': '∇U'}
Number of feature_functions: 27
(0)	tr(S)
(1)	tr(S∘S)
(2)	tr(S∘∇U)
(3)	tr(∇U∘∇U)
(4)	tr(∇U∘∇Uᵀ)
(5)	tr(S∘S∘S)
(6)	tr(S∘S∘∇U)
(7)	tr(S∘∇U∘∇U)
(8)	tr(S∘∇U∘∇Uᵀ)
(9)	tr(S∘∇Uᵀ∘∇U)
(10)	tr(∇U∘∇U∘∇U)
(11)	tr(∇U∘∇U∘∇Uᵀ)
(12)	tr(S∘S∘S∘S)
(13)	tr(S∘S∘S∘∇U)
(14)	tr(S∘S∘∇U∘∇U)
(15)	tr(S∘S∘∇U∘∇Uᵀ)
(16)	tr(S∘S∘∇Uᵀ∘∇U)
(17)	tr(S∘∇U∘S∘∇U)
(18)	tr(S∘∇U∘S∘∇Uᵀ)
(19)	tr(S∘∇U∘∇U∘∇U)
(20)	tr(S∘∇U∘∇U∘∇Uᵀ)
(21)	tr(S∘∇U∘∇Uᵀ∘∇U)
(22)	tr(S∘∇Uᵀ∘∇U∘∇U)
(23)	tr(∇U∘∇U∘∇U∘∇U)
(24)	tr(∇U∘∇U∘∇U∘∇Uᵀ)
(25)	tr(∇U∘∇U∘∇Uᵀ∘∇Uᵀ)
(26)	tr(∇U∘∇Uᵀ∘∇U∘∇Uᵀ)

Next we will further increase the complexity by 'tensorifying' the library with itself.

In [34]:
library_trace = library_trace + library_trace*library_trace
#library_trace

Now it is time to combine our matrix library and scalar library.

In [35]:
library = library + library_trace*library
len(library)

52233

We end up with a gigantic library we certainly want to trim, so we do by specifying the maximum degree in each variable:

In [36]:
library.trim({'S':2, '∇U':1, '∇Uᵀ':1})

Type of feature output: matrix
Symmetry of features: symmetric
Variables names: {'S', '∇U', '∇Uᵀ'}
Variables transpose map: {'S': 'S', '∇U': '∇Uᵀ', '∇Uᵀ': '∇U'}
Number of feature_functions: 68
(0)	I
(1)	S
(2)	∇U + (∇U)ᵀ
(3)	S∘S
(4)	S∘∇U + (S∘∇U)ᵀ
(5)	S∘∇Uᵀ + (S∘∇Uᵀ)ᵀ
(6)	∇U∘∇Uᵀ
(7)	∇Uᵀ∘∇U
(8)	S∘S∘∇U + (S∘S∘∇U)ᵀ
(9)	S∘S∘∇Uᵀ + (S∘S∘∇Uᵀ)ᵀ
(10)	S∘∇U∘S + (S∘∇U∘S)ᵀ
(11)	S∘∇U∘∇Uᵀ + (S∘∇U∘∇Uᵀ)ᵀ
(12)	S∘∇Uᵀ∘∇U + (S∘∇Uᵀ∘∇U)ᵀ
(13)	∇U∘S∘∇Uᵀ
(14)	∇Uᵀ∘S∘∇U
(15)	S∘S∘∇U∘∇Uᵀ + (S∘S∘∇U∘∇Uᵀ)ᵀ
(16)	S∘S∘∇Uᵀ∘∇U + (S∘S∘∇Uᵀ∘∇U)ᵀ
(17)	S∘∇U∘S∘∇Uᵀ + (S∘∇U∘S∘∇Uᵀ)ᵀ
(18)	S∘∇U∘∇Uᵀ∘S
(19)	S∘∇Uᵀ∘S∘∇U + (S∘∇Uᵀ∘S∘∇U)ᵀ
(20)	S∘∇Uᵀ∘∇U∘S
(21)	∇U∘S∘S∘∇Uᵀ
(22)	∇Uᵀ∘S∘S∘∇U
(23)	tr(S)I
(24)	tr(S)S
(25)	tr(S)(∇U + (∇U)ᵀ)
(26)	tr(S)(S∘∇U + (S∘∇U)ᵀ)
(27)	tr(S)(S∘∇Uᵀ + (S∘∇Uᵀ)ᵀ)
(28)	tr(S)∇U∘∇Uᵀ
(29)	tr(S)∇Uᵀ∘∇U
(30)	tr(S)(S∘∇U∘∇Uᵀ + (S∘∇U∘∇Uᵀ)ᵀ)
(31)	tr(S)(S∘∇Uᵀ∘∇U + (S∘∇Uᵀ∘∇U)ᵀ)
(32)	tr(S)∇U∘S∘∇Uᵀ
(33)	tr(S)∇Uᵀ∘S∘∇U
(34)	tr(S∘S)I
(35)	tr(S∘S)(∇U + (∇U)ᵀ)
(36)	tr(S∘S)∇U∘∇Uᵀ
(37)	tr(S∘S)∇Uᵀ∘∇U
(38)	tr(S∘∇U)I
(39)	tr(S∘∇U)S
(40)	tr(S∘

## Evaluate precursors
For each scenario in the dataset, we evalulate the precursor features and store it in a list.

In [39]:
precursor_dict = {}
for name, data in dataset.items():
    precursors = []
    for feature in library.feature_functions:
        precursors.append(feature(data))
    precursor_dict[name] = precursors

248